# Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

Here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.


## Import statements

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm as tqdm

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd

import random
import math
import time
import json
from tqdm import tqdm as tqdm

Set the random seeds for reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# downloading spacy english and italian
# !python -m spacy download en_core_web_sm

loading english sapcy models

In [5]:
spacy = spacy.load("en_core_web_sm")

## Previewing the Dataset
Link to the dataset https://www.kaggle.com/thoughtvector/customer-support-on-twitter

In [6]:
twitter_df = pd.read_csv("./../datasets/Customer Support Datasets for Chatbot Training/customer_support_twitter/twcs/twcs.csv")
twitter_df.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [7]:
twitter_df.shape

(2811774, 7)

Creating a list out of it, where each set has the inbound mail to company as well as it's response

In [ ]:
# dictionary of inbound tweets with tweet_id
inbound_reply_tweets = {}
for idx in tqdm(twitter_df.index):
    if not twitter_df.iloc[idx]["inbound"]:
        inbound_reply_tweets[twitter_df.iloc[idx]["tweet_id"]] = twitter_df.iloc[idx]["text"]

 16%|█▌        | 441317/2811774 [00:58<05:05, 7764.27it/s]

In [ ]:
conversation_pairs = []

for idx in tqdm(twitter_df.index):
    if twitter_df.iloc[idx]["inbound"]:
        try:
            inbound_to_company_tweet = twitter_df.iloc[idx]["text"]
            
            reply_tweet = inbound_reply_tweets[int(twitter_df.iloc[idx]["response_tweet_id"])]
            conversation_pairs.append([inbound_to_company_tweet, reply_tweet])
        except:
            pass

In [ ]:
len(conversation_pairs)

In [19]:
conversation_pairs[10]

['My picture on @Ask_Spectrum pretty much every day. Why should I pay $171 per month? https://t.co/U6ptkQa5Ik',
 '@115718 I apologize for the inconvenience. I will be glad to assist you. Can you DM me your Name and acct # or phone #? -JB']

## Defining the Fields

Question and Answer are standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [20]:
Question = Field(sequential = True,
                      tokenize = 'spacy',
#                       batch_first =True,
                      include_lengths=True,
                      init_token = '<sos>',
                      eos_token = '<eos>',
                      lower = True)

Answer = Field(sequential = True,
                    tokenize ='spacy',
#                     batch_first =True,
                    include_lengths=True,
                    init_token = '<sos>',
                    eos_token = '<eos>',
                    lower = True)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the list quescontext_ans_pairs

In [21]:
fields = [('questions', Question),('answers',Question)]

In [22]:
example = [data.Example.fromlist([conversation_pairs[i][0],conversation_pairs[i][1]], fields) for i in tqdm(range(len(conversation_pairs)))] 

100%|██████████| 1016105/1016105 [04:24<00:00, 3834.84it/s]


## Creating dataset

In [23]:
# creating dataset
twcsDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [24]:
(train, valid, test) = twcsDataset.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [25]:
(len(train), len(valid), len(test))

(711274, 152415, 152416)

An example from the dataset:

In [26]:
vars(train.examples[10])

{'questions': ['hey',
  '@116068',
  'i',
  'like',
  'your',
  'service',
  'but',
  'your',
  'sales',
  'rep',
  'that',
  'just',
  'called',
  'me',
  'to',
  '"',
  'consult',
  '"',
  'about',
  'my',
  'sites',
  'was',
  'rude',
  ',',
  'condescending',
  ',',
  'and',
  'would',
  "n't",
  'answer',
  'my',
  'questions',
  '.'],
 'answers': ['@558851',
  'i',
  "'m",
  'sorry',
  'to',
  'hear',
  'that',
  'jessica',
  '.',
  'could',
  'you',
  'please',
  'dm',
  'us',
  'your',
  'customer',
  'number',
  'and',
  'more',
  'details',
  '?',
  'i',
  "'d",
  'like',
  'to',
  'look',
  'into',
  'this',
  '.',
  '^lisi',
  'https://t.co/ryq25s2pfj']}

## Data Augmentation
copied from the repository of Woncoh1 https://github.com/woncoh1/END/blob/main/Assignment_Session_7_Sentiment_Analysis_using_LSTM_RNN.ipynb

In [29]:
# import libraries and prepare sentence text

import random
# for back translation
# import google_trans_new
# from google_trans_new import google_translator
# for word tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/deep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Back Translation

In [30]:
# translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1):
    # do nothing with probability of 1-p
    if random.uniform(0,1) > p:
        return sentence
    
    # combine tokenized sentence into one string
    sentence = " ".join(sentence)
    
    # instantiate translator
    translator = google_translator()
    
    # choose a target language
    available_langs = list(google_trans_new.LANGUAGES.keys())
    trans_lang = random.choice(available_langs)
    
    # translate to the target language
    translations = translator.translate(sentence, lang_tgt=trans_lang)
    
    # translate back to original language
    translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt="en")
    
    # select only one translation
    if len(translations_en_random) > 1:
        translations_en_random = translations_en_random[0]
        
    return word_tokenize(translations_en_random)

#### Random Deletion

In [31]:
# randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

#### Random Swap


In [32]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
        return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

#### Carry Out Data Augmentation

In [33]:
for example in tqdm(train.examples): 
#     example.questions = back_translate(example.questions, p=0.01)
    example.questions = random_deletion(example.questions, p=0.1)
    example.questions = random_swap(example.questions, n=1)
    
#     example.answers = back_translate(example.answers, p=0.01)
    example.answers = random_deletion(example.answers, p=0.01)
    example.answers = random_swap(example.answers, n=1)

## Building Vocabulary

In [34]:

MAX_VOCAB_SIZE = 25000
''' Use max of 25000 frequently occuring words to create vocabulary. Use pretrained fasttext embedding. Embedding of words that exist in the data but not in the
dictionary are set using normal distribution.
'''
Question.build_vocab(train,
                     max_size = MAX_VOCAB_SIZE,
                     vectors='fasttext.simple.300d',
                     unk_init = torch.Tensor.normal_,
                     min_freq = 2)


In [35]:
# get the vocab instance
vocab = Question.vocab

In [36]:
embedding_vector = Question.vocab.vectors
embedding_vector.shape

torch.Size([25004, 300])

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [37]:
print('Size of query vocab : ', len(Question.vocab))
# print('Size of answer vocab : ', len(Answer.vocab))
print('Top 10 words appeared in contex+question:', list(Question.vocab.freqs.most_common(10)))
# print("Top 10 words appeared in answer:", list(Answer.vocab.freqs.most_common(10)))

Size of query vocab :  25004
Top 10 words appeared in contex+question: [('.', 1371727), ('to', 839165), ('the', 707315), ('you', 700394), (',', 643591), ('i', 552709), ('we', 479521), ('?', 443268), ('a', 437916), ('!', 429916)]


## Create the iterators


In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
BATCH_SIZE = 8
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test),
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.questions),
                                                            sort_within_batch=True,
                                                            device = device)

In [40]:
vars(Question.vocab)

{'freqs': Counter({'seràn': 1,
          'cosas': 89,
          'sabemos': 22,
          'mi': 2718,
          'iphone': 28079,
          'empezò': 1,
          'dar': 135,
          'problemas': 311,
          'pronto': 139,
          'anunciaron': 1,
          'el': 6863,
          'iphonex': 877,
          '?': 443268,
          'ya': 3256,
          'o': 4715,
          'truquito': 1,
          '@115858': 22009,
          '@115940': 4736,
          'please': 218505,
          'add': 7412,
          '#': 120180,
          'nflredzone': 6,
          '@9833': 19,
          '@331517': 4,
          'to': 839165,
          'live': 8173,
          'tv': 8412,
          '.': 1371727,
          'this': 299295,
          'will': 86291,
          '@196559': 11,
          'it': 271580,
          'notch': 202,
          'of': 168571,
          'the': 707315,
          'other': 16454,
          'guys': 14829,
          'oh': 16212,
          'and': 396126,
          'maybe': 3509,
          'mak

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [41]:
class Encoder(nn.Module):
    '''
    encoder rakes ainpuyt and runs it through the lstm
    for example let's say the input size is [30,128],
    here 30 means 30 words/onehot and 128 means batch size
    
    first pass it through the embedding dim of let's say 512
    so the embedding output will bw [30,128,512]
    then pass this to lstm which will give you output and hidden cell state
    outputs wil have output at each time step so the first dim is sec_len
    hidden and cell is the final state output so they don't have src_len in the output 
    let's say hidden dim for lstm is 10
    outputs = [src_len,batch_size,hid_dim*n_layers]
    hidden = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer
    cell = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer
    
    '''
    
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = 1
        
        self.embedding = nn.Embedding(input_dim,emb_dim) #no dropout as only one layer!
        self.rnn = nn.GRU(emb_dim,hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src_len, batch_size, emb_dim]
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src_len, batch_size, emb_dim]
        outputs, hidden = self.rnn(embedded)  #no cell state because we are using GRU
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        # outputs are always from top hidden layer
        
        return hidden
        


## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [42]:
class Decoder(nn.Module):
    '''
        Decoder takes an input and runs it through the lstm
        decoder runs one word of batch
        for example let's say the input size is [128]
        here 128 means a batch of 128 words lets say all the 0th index 128 words
        
        first pass it through embedding dim of let's say 512
        so the embedding output will be [1,batch_size,emb_dim][1,128,512]
        then pass this ti lstm which will give you output and hidden cell state 
        outputs will have output at each time step so the first dim is src_len
        but since decoder has only one word of batch the first dim is 0 here
        hidden and cell os the dinal state output so they don't have src_len in the output as usual
        let's say hidden dim for lstm is 10
        outputs = [src_len,btach_size,hid_dim*n_directions][1,128,10] 1 here because decoder runs one word
        hidden = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer, we are using 1 here
        cell = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer, we are using 1 here
    '''
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)

        self.fc_out = nn.Linear(emb_dim + hid_dim * 2,
                                output_dim)  # we are using same dim here

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, context):
        
        # input = [batch_size]
        # hidden = [n_layers*n_direction,batch_size,hid_dim]
        # context = [n_layers*n_direction,batch_size,hid_dim]
        # n layers and n directions in the decoder will both always be 1,therefore ?? why always, only in this case, for example if we use 2 layers it may change
        # hidden = [1, batch_size, hid_dim]
        # context = [1, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input = [1, batch_size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1,batch_size,emb_dim]
        
        emb_con = torch.cat((embedded, context),
                             dim=2)  # last dim is output dim
        # emb_con = [1,batch_size,emb_dim+hid_dim]

        output, hidden = self.rnn(emb_con, hidden)
        # output = [seq_len,batch_size,hid_dim*n_directions]
        # hidden = [n_layers * n_directions,batch_size,hid_dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
        
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        
        # print('--------- check the shapes inside Decoder forward ---------')
        # print('input shape {}, hidden shape {}, context shape {}'.format(input.shape,hidden.shape,context.shape))
        # print('embedded shape {},emb_con shape {},output shape {},hidden shape {},'.format(embedded.shape,emb_con.shape,output.shape,hidden.shape))
        # print('prediction shape {}'.format(prediction.shape))
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [43]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        # src = [src_len,batch_size]
        # trg = [trg_len,batch_size]

        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

        # last hidden state of the encoder is the context
        context = self.encoder(src)

        # context also used as the initial hidden state of the decoder
        hidden = context

        # first inout to the decoder is the <sos> tokens
        input = trg[0, :]

        # print('--------- check the shapes inside Seq2Seq forward ---------')
        # print('src shape {}, trg shape {}'.format(src.shape,trg.shape))
        # print('batch_size is {} , trg_len is {} , trg_vocab_size {} , outputs shape {} ,'.format(batch_size,trg_len,trg_vocab_size,outputs.shape))
        # print('context vector {}, '.format(context.shape))
        
        for t in range(1, trg_len):

            # insert input token embeddingm previous hidden state and the context state
            # recieve output tensor i.e predictions and the new hidden state
            # context/hidden dim is [1,batch_size,hid_dim_enc], 1 for 1 layer
            # output dim [batch_size,vocab_len]
            
            output, hidden = self.decoder(input, hidden, context)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest prediction token from out predictions
            # top1 has the dim [batch_size,vocab_len]
            # each example/word output a vector of target vocab length and we are taking the index of that
            # out has trg_vocab size , let's say there are 1000 words and by taking index
            # we are saying select the word which has highest value
            top1 = output.argmax(
                1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [44]:
len(Question.vocab)

25004

In [46]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Question.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [47]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25004, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(25004, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=25004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 47,984,044 trainable parameters


We create an optimizer.

In [49]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [50]:
ANSWER_PAD_IDX = Question.vocab.stoi[Question.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = ANSWER_PAD_IDX)

We then create the training loop...

In [51]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [52]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best 


In [55]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'twcs_test.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('twcs_vanila_encoder_decoder.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')